In [294]:
%load_ext autoreload
%autoreload 2

import sys, os
from pathlib import Path
import plotly.graph_objects as go
from typing import Iterable
import pandas as pd

# Fix import
sys.path.append(os.path.abspath(os.path.join('..')))
from facilitators import BarFacilitator, StackedQuantityEvolutionFacilitatorBase, StackedQuantityEvolutionFacilitatorBase2
from data import read_sol_keys, AGGREGATION, aggregator
from data import data_processor as dp
from facilitators import my_color_generator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [295]:
PROJECT_ROOT_DIR = Path().resolve().parent
SOLUTION_DIR = PROJECT_ROOT_DIR / "solution_files3"

#EXPENSIVE_HLDH = SOLUTION_DIR / "solution_seasonal_free_3_d40_h6_s6_6.sol"
EXPENSIVE_HLDH = SOLUTION_DIR / "solution_seasonal_10HLDH_d40_h6_s6_6.sol"
SEASON_DATA = SOLUTION_DIR / "solution_seasonal_d40_h6_s6_6.sol"

WRITE_DIR = PROJECT_ROOT_DIR / "generated_figs3"
WRITE_FIGS = True

REGION = "ALL"
YEAR = 2050

In [296]:
def stacked_accumulated_energy(sol_path: str, region: str = "ALL", title_addon: str = None, write_fig = False) -> go.Figure:
    storage_evolution = StackedQuantityEvolutionFacilitatorBase2(sol_path=sol_path, type_of_data_to_read="NewStorageCapacity", extra_identifying_columns=["Region"])

    d = storage_evolution.get_relevant_data()
    #d.show_example_rows(5)
    d.filter_by_containing_string(column="Storage", identifier="S_Heat_HLDH")

    #if region == "ALL":
        #d.aggreagate_all_by_sum(column_to_aggregate="Region", aggregated_entry_name=region, column_to_sum="Value")

    # Remove those who are 0 for all years
    techs_over_zero = d.df[d.df["Value"] > 0]["Region"].unique()
    d.filter_by_list(column="Region", identifier_list=techs_over_zero)

    # Do cumsum over years
    d.df["Value"] = d.df.groupby("Region")["Value"].cumsum()

    # Keep the 5 biggest regions in 2050
    top_regions = d.df[d.df["Year"] == YEAR].nlargest(5, "Value")["Region"].values
    bottom_regions = set(d.df["Region"].unique()) - set(top_regions)
    bottom_df = d.df[d.df["Region"].isin(bottom_regions)]

    # Aggregate the bottom regions
    bottom_df["Region"] = "Others"
    bottom_df = bottom_df.groupby([col for col in bottom_df.columns if col != "Value"], as_index=False).sum(numeric_only=True)

    d.filter_by_list(column="Region", identifier_list=top_regions)
    d.df = pd.concat([d.df, bottom_df], ignore_index=True)
    
    traces = storage_evolution.generate_traces(d)


    # Sort the traces by the value in 2050
    traces = sorted(traces, key=lambda x: x.y[-1], reverse=True)

    # But put the "Others" at the end
    traces = [x for x in traces if x.name == "Others"] + [x for x in traces if x.name != "Others"]

    fig = storage_evolution.plot(traces)

    fig.update_layout(
        width=1400,
        height=800,
        title=f"Accumulated Storage Capacity District Heat Europe",
        yaxis_title="TWh",
        xaxis_title="Year",
    )

    if write_fig:
        fig.write_image(WRITE_DIR / f"stacked_accumulated_storage_energy_region_{region}_{title_addon}.png")

    return fig

In [281]:
#stacked_accumulated_energy(DAILY_DATA, region=REGION, title_addon="Daily", write_fig=WRITE_FIGS)[0].show()
stacked_accumulated_energy(SEASON_DATA, region=REGION, title_addon="", write_fig=WRITE_FIGS).show()
stacked_accumulated_energy(EXPENSIVE_HLDH, region=REGION, title_addon="Expensive", write_fig=WRITE_FIGS).show()

Converting from PetaJoules to TerraWattHours


C:\Users\mortensv\AppData\Local\Temp\ipykernel_29752\1857296959.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Converting from PetaJoules to TerraWattHours


C:\Users\mortensv\AppData\Local\Temp\ipykernel_29752\1857296959.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



# Pie plots for electricity / power

In [297]:
def power_pie_plot(sol_path: str) -> go.Figure:

    # Pie plot for electronics
    data_prod_by_tech = dp.DataProcessor(sol_paths={"anonym": sol_path}, type_of_data_to_read="ProductionByTechnologyAnnual")
    data_p = dp.DataProcessor(sol_paths={"anonym": sol_path}, type_of_data_to_read="TotalCapacityAnnual")

    # Look at 2050
    data_p.filter_by_identifier(column="Year", identifier=2050)

    # Remove all that starts with "D_"
    data_p.df = data_p.df[~data_p.df["Technology"].str.startswith("D_")]

    # Keep those over 0
    data_p.df = data_p.df[data_p.df["Value"] > 0]

    # Aggregate by technology according to the aggregation
    #print("Length before aggregation", len(data_p.df))
    #data_p.aggregate_by_sum(column_to_sum="Value", groups_memberships=AGGREGATION)
    #data_prod_by_tech.show_example_rows(5)
    #print(data_prod_by_tech.df["Fuel"].unique())
    #data_prod_by_tech.filter_by_identifier(column="Fuel", identifier="Heat_District")
    #data_prod_by_tech.filter_by_identifier(column="Fuel", identifier="Heat_Low_Building")
    #data_prod_by_tech.filter_by_identifier(column="Fuel", identifier="L")
    #data_prod_by_tech.show_example_rows(5)
    data_prod_by_tech.filter_by_identifier(column="Fuel", identifier="Power")
    data_prod_by_tech.df = data_prod_by_tech.df[data_prod_by_tech.df["Value"] > 0]
    #data_prod_by_tech.show_example_rows(5)
    techs = list(data_prod_by_tech.df["Technology"].unique())

    data_p.filter_by_list(column="Technology", identifier_list=techs)

    data_p.aggregate_by_sum_function(column_to_change="Technology", column_to_sum="Value", function=aggregator)

    # Create color list
    color_list = [my_color_generator(tech) for tech in data_p.df["Technology"]]

    # Make a pie plot
    fig = go.Figure(data=[go.Pie(labels=data_p.df["Technology"], values=data_p.df["Value"], 
                                 textposition='inside', textinfo='percent+label',
                                 textfont=dict(size=24),
                                 marker=dict(colors=color_list))])

    # Loop labels, and change the color of the pie
    #color_gen = my_color_generator()
    #for i, label in enumerate(fig.data[0].labels):
    #    fig.data[0].marker.colors += [my_color_generator(label)]

    fig.update_layout(
        title="Power Capacity (Energy / Time) 2050",
        width=900,
        height=800
    )

    return fig

In [298]:
fig = power_pie_plot(SEASON_DATA)
if WRITE_FIGS:
    fig.write_image(WRITE_DIR / "power_pie_plot.png")
fig.show()

fig = power_pie_plot(EXPENSIVE_HLDH)
if WRITE_FIGS:
    fig.write_image(WRITE_DIR / "power_pie_plot_expensive.png")
fig.show()

Converting from PetaJoules to TerraWattHours
No unit conversion applied!


Converting from PetaJoules to TerraWattHours
No unit conversion applied!


In [277]:
def power_pie_plot2(sol_path: str) -> go.Figure:

    # Pie plot for electronics
    data_prod_by_tech = dp.DataProcessor(sol_paths={"anonym": sol_path}, type_of_data_to_read="ProductionByTechnologyAnnual")
    #data_p = dp.DataProcessor(sol_paths={"anonym": sol_path}, type_of_data_to_read="TotalCapacityAnnual")

    # Look at 2050
    #data_p.filter_by_identifier(column="Year", identifier=2050)
    data_prod_by_tech.filter_by_identifier(column="Year", identifier=2050)

    # Remove all that starts with "D_"
    #data_p.df = data_p.df[~data_p.df["Technology"].str.startswith("D_")]
    data_prod_by_tech.df = data_prod_by_tech.df[~data_prod_by_tech.df["Technology"].str.startswith("D_")]

    # Keep those over 0
    #data_p.df = data_p.df[data_p.df["Value"] > 0]

    # Aggregate by technology according to the aggregation
    #print("Length before aggregation", len(data_p.df))
    #data_p.aggregate_by_sum(column_to_sum="Value", groups_memberships=AGGREGATION)
    #data_prod_by_tech.show_example_rows(5)
    #print(data_prod_by_tech.df["Fuel"].unique())
    #data_prod_by_tech.filter_by_identifier(column="Fuel", identifier="Heat_District")
    #data_prod_by_tech.filter_by_identifier(column="Fuel", identifier="Heat_Low_Building")
    data_prod_by_tech.filter_by_identifier(column="Fuel", identifier="Heat_Low_Industrial")
    #data_prod_by_tech.show_example_rows(5)
    #data_prod_by_tech.filter_by_identifier(column="Fuel", identifier="Power")
    data_prod_by_tech.df = data_prod_by_tech.df[data_prod_by_tech.df["Value"] > 0]
    #data_prod_by_tech.show_example_rows(5)
    #techs = list(data_prod_by_tech.df["Technology"].unique())

    #data_p.filter_by_list(column="Technology", identifier_list=techs)

    #data_prod_by_tech.aggregate_by_sum_function(column_to_change="Technology", column_to_sum="Value", function=aggregator)

    # Create color list
    color_list = [my_color_generator(tech) for tech in data_prod_by_tech.df["Technology"]]

    # Make a pie plot
    fig = go.Figure(data=[go.Pie(labels=data_prod_by_tech.df["Technology"], values=data_prod_by_tech.df["Value"], 
                                 textposition='inside', textinfo='percent+label',
                                 textfont=dict(size=20),
                                 marker=dict(colors=color_list))])

    # Loop labels, and change the color of the pie
    #color_gen = my_color_generator()
    #for i, label in enumerate(fig.data[0].labels):
    #    fig.data[0].marker.colors += [my_color_generator(label)]

    fig.update_layout(
        title="Energy Production 2050",
        width=900,
        height=800
    )

    return fig

In [278]:
fig = power_pie_plot2(SEASON_DATA)
fig.show()

if WRITE_FIGS:
    fig.write_image(WRITE_DIR / "power_pie_plot2.png")

fig = power_pie_plot2(EXPENSIVE_HLDH)
fig.show()

if WRITE_FIGS:
    fig.write_image(WRITE_DIR / "power_pie_plot2_expensive.png")

Converting from PetaJoules to TerraWattHours


Converting from PetaJoules to TerraWattHours


In [269]:
# Load Par_SpecifiedAnnualDemand.csv

def heat_demand_pie_plot(demand_data: pd.DataFrame) -> go.Figure:

    # Only 2050
    demand_data = demand_data[demand_data["Year"] == 2050]

    # Only heat
    """demand_data = demand_data[demand_data["Fuel"].str.contains("Heat")]

    # Delete Heat from the name
    demand_data["Fuel"] = demand_data["Fuel"].apply(lambda x: x.replace("Heat_", "").strip())

    # Group all industrial
    demand_data["Fuel"] = demand_data["Fuel"].apply(lambda x: "Industrial" if "Industrial" in x else x)

    # Make Low_Building into Building
    demand_data["Fuel"] = demand_data["Fuel"].apply(lambda x: "Building" if "Low_Building" in x else x)"""

    # Group all heats
    demand_data["Fuel"] = demand_data["Fuel"].apply(lambda x: "Heat" if ("Heat" in x) or ("Cool" in x) else x)

    # Group others
    demand_data["Fuel"] = demand_data["Fuel"].apply(lambda x: "Others" if "Heat" not in x else x)

    # Group by fuel and sum
    demand_data = demand_data.groupby("Fuel", as_index=False).sum(numeric_only=True)

    # Make a pie plot
    fig = go.Figure(data=[go.Pie(labels=demand_data["Fuel"], values=demand_data["Value"], 
                                    textposition='inside', textinfo='percent+label',
                                    textfont=dict(size=24),
                                    marker=dict(colors=[my_color_generator(fuel) for fuel in demand_data["Fuel"]]))])

    fig.update_layout(
        title="Demand 2050",
        width=900,
        height=800
    )

    return fig




In [270]:
#demand_data = pd.read_csv(SOLUTION_DIR / "Par_SpecifiedAnnualDemand.csv")
demand_data = pd.read_excel(SOLUTION_DIR / "Parameter_seasonal.xlsx", sheet_name="Par_SpecifiedAnnualDemand")

fig = heat_demand_pie_plot(demand_data)

if WRITE_FIGS:
    print("Hei")
    fig.write_image(WRITE_DIR / "heat_demand_2050_pie_plot.png")

fig.show()

# Read demand data from sheet in excel

demand_data = pd.read_excel(SOLUTION_DIR / "Parameter_seasonal_District35.xlsx", sheet_name="Par_SpecifiedAnnualDemand")

heat_demand_pie_plot(demand_data).show()


Hei


C:\Users\mortensv\AppData\Local\Temp\ipykernel_29752\1243851363.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\mortensv\AppData\Local\Temp\ipykernel_29752\1243851363.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



C:\Users\mortensv\AppData\Local\Temp\ipykernel_29752\1243851363.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\mortensv\AppData\Local\Temp\ipykernel_29752\1243851363.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

